<a href="https://colab.research.google.com/github/hernandezurbina/RLAdventures/blob/master/TD3_L17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pybullet

     |████████████████████████████████| 95.3MB 55kB/s 


In [2]:
import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import pybullet_envs
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
from gym import wrappers
from torch.autograd import Variable
from collections import deque

In [3]:
class ReplayBuffer(object):
  def __init__(self, max_size=1e6):
    self.storage = []
    self.max_size = max_size
    self.ptr = 0

  def add(self, transition):
    if len(self.storage) == self.max_size:
      self.storage[int(self.ptr)] = transition
      self.ptr = (self.ptr + 1) % self.max_size
    else:
      self.storage.append(transition)
  
  def sample(self, batch_size):
    ind = np.random.randint(0, len(self.storage), size=batch_size)
    batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = [], [], [], [], []
    for i in ind:
      state, next_state, action, reward, done = self.storage[i]
      batch_states.append(np.array(state, copy=False))
      batch_next_states.append(np.array(next_state, copy=False))
      batch_actions.append(np.array(action, copy=False))
      batch_rewards.append(np.array(reward, copy=False))
      batch_dones.append(np.array(done, copy=False))
    return np.array(batch_states), np.array(batch_next_states), np.array(batch_actions), np.array(batch_rewards).reshape(-1, 1), np.array(batch_dones).reshape(-1, 1)

In [4]:
class Actor(nn.Module):
  def __init__(self, state_dim, action_dim, max_action):
    super(Actor, self).__init__()
    self.layer_1 = nn.Linear(state_dim, 400)
    self.layer_2 = nn.Linear(400, 300)
    self.layer_3 = nn.Linear(300, action_dim)
    self.max_action = max_action

  def forward(self, x):
    x = F.relu(self.layer_1(x))
    x = F.relu(self.layer_2(x))
    x = self.max_action * torch.tanh(self.layer_3(x))
    return x

In [5]:
class Critic(nn.Module):
  def __init__(self, state_dim, action_dim):
    super(Critic, self).__init__()
    self.layer_1 = nn.Linear(state_dim + action_dim, 400)
    self.layer_2 = nn.Linear(400, 300)
    self.layer_3 = nn.Linear(300, 1)
    # second critic NN:
    self.layer_4 = nn.Linear(state_dim + action_dim, 400)
    self.layer_5 = nn.Linear(400, 300)
    self.layer_6 = nn.Linear(300, 1)

  def forward(self, x, u):
    xu = torch.cat([x, u], 1)
    x1 = F.relu(self.layer_1(xu))
    x1 = F.relu(self.layer_2(x1))
    x1 = self.layer_3(x1)
    # second critic NN:
    x2 = F.relu(self.layer_4(xu))
    x2 = F.relu(self.layer_5(x2))
    x2 = self.layer_6(x2)
    return x1, x2
  
  def Q1(self, x, u):
    xu = torch.cat([x, u], 1)
    x1 = F.relu(self.layer_1(xu))
    x1 = F.relu(self.layer_2(x1))
    x1 = self.layer_3(x1)
    return x1

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class TD3(object):
  def __init__(self, state_dim, action_dim, max_action):
    self.actor = Actor(state_dim, action_dim, max_action).to(device)
    self.actor_target = Actor(state_dim, action_dim, max_action).to(device)
    self.actor_target.load_state_dict(self.actor.state_dict())
    self.actor_optimizer = torch.optim.Adam(self.actor.parameters())

    self.critic = Critic(state_dim, action_dim).to(device)
    self.critic_target = Critic(state_dim, action_dim).to(device)
    self.critic_target.load_state_dict(self.critic.state_dict())
    self.critic_optimizer = torch.optim.Adam(self.critic.parameters())

    self.max_action = max_action

  def select_action(self, state):
    state = torch.Tensor(state.reshape(1, -1)).to(device)
    return self.actor(state).cpu().data.numpy().flatten()

  def train(self, replay_buffer, iterations, batch_size=100, discount=0.99, tau=0.005, policy_noise=0.2, noise_clip=0.5, policy_freq=2):
    for it in range(iterations):
      batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = replay_buffer.sample(batch_size)
      state = torch.Tensor(batch_states).to(device)
      next_state = torch.Tensor(batch_next_states).to(device)
      action = torch.Tensor(batch_actions).to(device)
      reward = torch.Tensor(batch_rewards).to(device)
      done = torch.Tensor(batch_dones).to(device)

      next_action = self.actor_target(next_state)

      noise = torch.Tensor(batch_actions).data.normal_(0, policy_noise).to(device)
      noise = noise.clamp(-noise_clip, noise_clip)
      next_action = (next_action + noise).clamp(-self.max_action, self.max_action)

      target_Q1, target_Q2 = self.critic_target(next_state, next_action)
      target_Q = torch.min(targetQ1, targetQ2)
      target_Q = reward + ((1 - done) * (discount * target_Q)).detach()

      current_Q1, current_Q2 = self.critic(state, action)
      critic_loss = F.mse_loss(current_Q1, target_Q) + F.mse_loss(current_Q2, target_Q)

      self.critic_optimizer.zero_grad()
      critic_loss.backward()
      self.critic_optimizer.step()

      if it % freq == 0:
        actor_loss = -self.critic.Q1(state, self.actor(state)).mean()
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
        
        for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
          target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)

        for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
          target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)          

    def save(self, filename, directory):
      torch.save(self.actor.state_dict(), '%s/%s_actor.pth' % (directory, filename))
      torch.save(self.critic.state_dict(), '%s/%s_critic.pth' % (directory, filename))
    def load(self, filename, directory):
      self.actor.load_state_dict(torch.load('%s/%s_actor.pth' % (directory, filename)))
      self.critic.load_state_dict(torch.load('%s/%s_critic.pth' % (directory, filename)))

In [7]:
def evaluate_policy(policy, eval_episodes=10):
  avg_reward = 0
  for _ in range(eval_episodes):
    obs = env.reset()
    done = False
    while not done:
      action = policy.select_action(np.array(obs))
      obs, reward, done, _ = env.step(action)
      avg_reward += reward
    avg_reward /= eval_episodes
    print('----------------------------------------')
    print('Average reward over the evaluation step: %f' % (avg_reward))
    print('----------------------------------------')
    return avg_reward

In [8]:
env_name = 'HalfCheetahBulletEnv-v0'
seed = 0
start_timesteps = 1e4
eval_freq = 5e3
max_timesteps = 5e5
save_models = True
expl_noise = 0.1
batch_size = 100
discount = 0.99
tau = 0.005
policy_noise = 0.2
noise_clip = 0.5
policy_freq = 2

In [9]:
file_name = "%s_%s_%s" % ('TD3', env_name, str(seed))
print('---------------------------')
print('Settings: %s' % (file_name))
print('---------------------------')

---------------------------
Settings: TD3_HalfCheetahBulletEnv-v0_0
---------------------------


In [10]:
if not os.path.exists('./results'):
  os.makedirs('./results')
if save_models and not os.path.exists('./pytorch_models'):
  os.makedirs('./pytorch_models')

In [11]:
env = gym.make(env_name)

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [12]:
env.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
max_action = float(env.action_space.high[0])

In [13]:
policy = TD3(state_dim, action_dim, max_action)

In [14]:
replay_buffer = ReplayBuffer()

In [15]:
evaluations = [evaluate_policy(policy)]

----------------------------------------
Average reward over the evaluation step: -143.135146
----------------------------------------


In [16]:
def mkdir(base, name):
  path = os.path.join(base, name)
  if not os.path.exists(path):
    os.makedirs(path)
  return path

work_dir = mkdir('exp','brs')
monitor_dir = mkdir(work_dir, 'monitor')
max_episode_steps = env._max_episode_steps
save_env_vid = False
if save_env_vid:
  env = wrappers.Monitor(env, monitor_dir, force=True)
  env.reset()

In [17]:
total_timesteps = 0
timesteps_since_eval = 0
episode_num = 0
done = True
t0 = time.time()

In [ ]:
while total_timesteps < max_timesteps:
  if done:
    if total_timesteps != 0:
      print('Total timesteps: {} Episode num: {} Reward: {}'.format(total_timesteps, episode_num, episode_reward))
      policy.train(replay_buffer, episode_timesteps, batch_size, discount, tau, policy_noise, noise_clip, policy_freq)

    if timesteps_since_eval >= eval_freq:
      timesteps_since_eval %= eval_freq
      evaluations.append(evaluate_policy(policy))
      policy.save(file_name, directory='./pytorch_models')
      np.save('./results/%s' % (file_name), evaluations)
    
    obs = env.reset()
    done = False

    episode_reward = 0
    episode_timesteps = 0
    episode_num += 1

    if total_timesteps < start_timesteps:
      action = env.action_space.sample()
    else:
      action = policy.select_action(np.array(obs))
      if expl_noise != 0:
        action = (action + np.random.normal(0, expl_noise, size=env.action_space.shape[0])).clip(env.action_space.low, env.action_space.high)

    new_obs, reward, done, _ = env.step(action)

    done_bool = 0 if episode_timesteps + 1 == env._max_episode_steps else float(done)

    episode_reward += reward

    replay_buffer.add((obs, new_obs, action, reward, done_bool))

    obs = new_obs
    episode_timesteps += 1
    total_timesteps += 1
    timesteps_since_eval += 1

evaluations.append(evaluate_policy(policy))
if save_models: policy.save('%s' % (file_name), directory='./pytorch_models')
np.save('./results/%s' % (file_name), evaluations)

